# Regressão Linear Múltipla



## 1. Introdução

Muitos algoritmos de aprendizagem de máquinas utilização métodos de otimização. Esses algoritmos são usados por algoritmos de aprendizado de máquina para encontrar um bom conjunto de parâmetros do modelo, dado um conjunto de dados de treinamento. O algoritmo de otimização mais comum usado na aprendizagem de máquinas é o gradiente descendente estocástico. Neste tutorial, você descobrirá como implementar uma gradiente descendente estocástico para otimizar um algoritmo de regressão linear.

Ao final desta aula voce estará apto a:

1. Estimar os coeficientes de regressão linear usando a descida gradiente estocástica.
2. Fazer previsões para regressão linear multivariada.
3. Implementar regressão linear com  gradiente descendente estocástico para fazer previsões em novos dados.

### 1.1 Dataset - Seguro de Veículo Sueco

Neste tutorial, usaremos o dataset que trata da Qualidade do Vinhos (Wine Quality Data), o qual pode permitir a previsão da qualidade do vinho branco, ajudando o especialista em vinhos na avaliação de qualidade. O RMSE de linha de base do problema é de aproximadamente 0.148 pontos de qualidade. O arquivo winequality-white.csv está disponível no diretório presente. 

### 1.2 Regressão Linear Multivariada

A regressão linear é uma técnica para prever um valor real. Confusamente, esses problemas em que um valor real deve ser previsto são chamados de problemas de regressão. A regressão linear é uma técnica em que uma linha reta é usada para modelar a relação entre valores de entrada e saída. Em mais de duas dimensões, esta linha reta pode ser pensada como um plano ou hiperplano.

As previsões são feitas como uma combinação dos valores de entrada para prever o valor de saída. Cada atributo de entrada (x) é ponderado usando um coeficiente (b), e o objetivo do algoritmo de aprendizagem é descobrir um conjunto de coeficientes que resulte em boas previsões (y).


![alt text](images/regressao_linear_multipla_modelo.png "")
                             
Os coeficientes podem ser encontrados usando gradiente descendente estocástico.

### 1.3 Gradiente Descendente Estocástico

Gradiente Descendente é o processo de minimização de uma função seguindo a inclinação ou gradiente dessa função. Na aprendizagem em máquina, podemos usar uma técnica que avalie e atualize os coeficientes de cada iteração chamada gradiente descendente estocástico para minimizar o erro de um modelo em nossos dados de treinamento.

A maneira como esse algoritmo de otimização funciona é que cada instância de treinamento é submetida uma de cada vez ao modelo. O modelo faz uma previsão para uma instância de treinamento, o erro é calculado e o modelo é atualizado para reduzir o erro para a próxima previsão. Este processo é repetido para um número fixo de iterações.

Esse procedimento pode ser usado para encontrar o conjunto de coeficientes em um modelo que resulte no menor erro para o modelo nos dados de treinamento. Cada iteração, os coeficientes *(b)* são atualizados usando a equação:


![alt text](images/coeficiente_b_gradiente.png "")


Onde *b* é o coeficiente ou o peso a ser otimizado, a *taxa de aprendizado (learning rate)* é uma taxa que você deve configurar (por exemplo, 0,01), o *erro* é o erro de predição para o modelo nos dados de treinamento atribuídos ao peso e *x* é o valor de entrada.


## 2. Tutorial

Este tutorial é dividido em 3 partes:

1. Fazendo Predições.
2. Coeficientes de estimativa. 
3. Estudo de caso de qualidade do vinho.

Isso proporcionará a base que você precisa implementar e aplicar uma regressão linear com descida descendente estocástica em seus próprios problemas de modelagem preditiva.

### 2.1 Fazer previsões

O primeiro passo é desenvolver uma função que possa fazer previsões. Isso será necessário tanto na avaliação dos valores dos coeficientes do candidato quanto na aplicação do gradiente descendente estocástico. Após o modelo ser finalizado, começaremos a fazer previsões em dados de teste ou novos dados. Abaixo está uma função chamada *predict ()* que prediz um valor de saída para uma linha, dado um conjunto de coeficientes.

O primeiro coeficiente é sempre a intercepção, também chamado de viés (em ingles, bias) ou b0, pois é independente e não é responsável por um valor de entrada específico.

In [58]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [59]:
# Make a prediction with coefficients
def predict(row, coefficients):
  yhat = coefficients[0]
  for i in range(len(row)-1):
    yhat += coefficients[i + 1] * row[i]
  return yhat

### Exemplo

Dado conjunto de dados abaixo, será apresentado o uso da função predict confome exemplo que segue.

x |  y
--| -
1 | 1
2 | 3
4 | 3
3 | 2
5 | 5

Neste exemplo, existe um único valor de entrada (x) e dois valores de coeficiente (b0 e b1). A equação de predição que modelamos para este problema é:
*y = b0 + b1 × x *

Ou, com os valores de coeficientes específicos, escolhemos à mão como:
y = 0,4 + 0,8 * x 

Ao executar esta função, obtemos previsões razoavelmente próximas da saída esperada (y) valores.

### 2.2 Estimativa de Coeficientes 

Para podermos estimar os valores dos coeficientes para nossos dados de treinamento usando gradiente descendente estocástico precisamos definir dois parâmetros:

1. Taxa de aprendizado: usado para limitar a quantidade que cada coeficiente é corrigido sempre que é atualizado.
2. Épocas: o número de vezes para percorrer os dados de treinamento ao atualizar os coeficientes.

Estes parametros, juntamente com os dados de treinamento, serão os argumentos para a função. Existem 3 loops que precisamos executar na função:

1. Loop em cada época.
2. Faça um loop sobre cada linha nos dados de treinamento para uma época.
3. Faça um loop sobre cada coeficiente e atualize-o para uma linha dos dados em uma época.

Como você pode ver, atualizamos cada coeficiente para cada linha nos dados de treinamento, em cada época. Os coeficientes são atualizados com base no erro que o modelo fez. O erro é calculado como a diferença entre a previsão feita com os coeficientes do candidato e o valor de saída esperado.

*error = prediction − expected*

Existe um coeficiente para ponderar cada atributo de entrada, e estes são atualizados de forma consistente, por exemplo:
 
b1 (t + 1) = b1 (t) - taxa de aprendizado x erro (t) x x1 (t)


O coeficiente especial no início da lista, também chamado de intercepção ou a polarização, é atualizado de forma semelhante, exceto sem uma entrada porque não está associado a um valor de entrada específico:

b0 (t + 1) = b0 (t) - taxa de aprendizagem * erro (t) 

Abaixo está uma função denominada coefficients_sgd() que calcula valores de coeficientes para um conjunto de dados de treinamento usando gradiente descendente estocástico.

Usamos uma pequena taxa de aprendizado de 0,001 e treinamos o modelo por 50 épocas, ou 50 exposições dos coeficientes para todo o conjunto de dados de treinamento. Ao executar o exemplo, imprime uma mensagem a cada época com o erro de soma quadrada para aquela época e o conjunto final de coeficientes.

In [66]:
# Estimate linear regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    print ('Coeficiente Inicial=' + str(coef))
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            sum_error += error**2
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] - l_rate * error * row[i] 
        print(('epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error)))
    return coef

In [100]:
def normalize(dataset):
    dataset_aux = dataset[:10]
    print(dataset_aux)
    dataset_values = dataset_aux.values
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(1, 10))
    dataset_scaled = min_max_scaler.fit_transform(dataset_values)
    df = pd.DataFrame(dataset_scaled)
    return df

In [109]:
dataset = pd.read_csv("winequality-white.csv", delimiter=";")
dataset.loc[0:10]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.70,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.60,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.90,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.50,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.50,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,8.1,0.28,0.40,6.90,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
6,6.2,0.32,0.16,7.00,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6
7,7.0,0.27,0.36,20.70,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
8,6.3,0.30,0.34,1.60,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
9,8.1,0.22,0.43,1.50,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6


In [110]:
df = normalize(train_dataset)
train_dataset = df.head(int(len(dataset)*0.8))
test_dataset = df.tail(int(len(dataset)*0.2))

          0     1          2          3          4          5          6   \
0   4.789474   5.5   7.666667  10.000000   1.642857   9.454545   8.382022   
1   1.473684   8.2   7.000000   1.046875   4.214286   1.000000   4.539326   
2  10.000000   6.4   9.000000   3.531250   4.857143   5.363636   1.000000   
3   5.736842   1.9   6.333333   4.281250  10.000000  10.000000  10.000000   
4   5.736842   1.9   6.333333   4.281250  10.000000  10.000000  10.000000   
5  10.000000   6.4   9.000000   3.531250   4.857143   5.363636   1.000000   
6   1.000000  10.0   1.000000   3.578125   1.642857   5.363636   4.943820   
7   4.789474   5.5   7.666667  10.000000   1.642857   9.454545   8.382022   
8   1.473684   8.2   7.000000   1.046875   4.214286   1.000000   4.539326   
9  10.000000   1.0  10.000000   1.000000   1.000000   4.818182   4.235955   

       7     8     9          10   11  
0  10.000   1.0   6.0   1.000000  1.0  
1   1.250  10.0  10.0   3.863636  1.0  
2   2.625   8.8   5.0   6.318182

In [81]:
l_rate = 0.001
n_epoch = 100

In [82]:
coeff = coefficients_sgd(train_dataset.values, l_rate, n_epoch)
print(coeff)

Coeficiente Inicial=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
epoch=0, lrate=0.001, error=7171.804
epoch=1, lrate=0.001, error=6833.875
epoch=2, lrate=0.001, error=6794.795
epoch=3, lrate=0.001, error=6757.905
epoch=4, lrate=0.001, error=6723.087
epoch=5, lrate=0.001, error=6690.762
epoch=6, lrate=0.001, error=6660.884
epoch=7, lrate=0.001, error=6633.275
epoch=8, lrate=0.001, error=6607.735
epoch=9, lrate=0.001, error=6584.075
epoch=10, lrate=0.001, error=6562.123
epoch=11, lrate=0.001, error=6541.726
epoch=12, lrate=0.001, error=6522.745
epoch=13, lrate=0.001, error=6505.058
epoch=14, lrate=0.001, error=6488.555
epoch=15, lrate=0.001, error=6473.135
epoch=16, lrate=0.001, error=6458.709
epoch=17, lrate=0.001, error=6445.199
epoch=18, lrate=0.001, error=6432.531
epoch=19, lrate=0.001, error=6420.640
epoch=20, lrate=0.001, error=6409.468
epoch=21, lrate=0.001, error=6398.961
epoch=22, lrate=0.001, error=6389.070
epoch=23, lrate=0.001, error=6379.752
epoch=24, lrate=0